In [29]:
# Cell 1: Setup and Imports (Same as before)
import numpy as np
import os
import glob
from tqdm import tqdm
import gc

print("🚀 CPH Dataset Processing - FIXED VERSION")
print("=" * 50)

# Check available data
print("📁 Checking dataset files...")
base_path = '/kaggle/input'

# Find your dataset directory
dataset_path = None
for root, dirs, files in os.walk(base_path):
    if any(f.endswith('.yuv') for f in files):
        dataset_path = root
        break

print(f"📁 Dataset path: {dataset_path}")
yuv_count = len([f for f in os.listdir(dataset_path) if f.endswith('.yuv')])
dat_count = len([f for f in os.listdir(dataset_path) if f.endswith('.dat')])
print(f"✅ Found {yuv_count} YUV files and {dat_count} DAT files")

🚀 CPH Dataset Processing - FIXED VERSION
📁 Checking dataset files...
📁 Dataset path: /kaggle/input/cph-intra-dataset
✅ Found 12 YUV files and 96 DAT files


In [30]:
# Cell 2: Updated CPH Processor - NO LIMITS VERSION
class FullCPHProcessor:
    def __init__(self):
        self.QP_VALUES = [22, 27, 32, 37]
        self.RESOLUTIONS = ['768x512', '1536x1024', '2880x1920', '4928x3264']
        self.input_path = dataset_path
        self.output_path = '/kaggle/working'
        
        # NO FRAME LIMITS - Process all available frames
        self.frame_limits = None
        
    def get_total_frames_in_yuv(self, filepath, width, height):
        """Calculate total number of frames available in YUV file"""
        try:
            file_size = os.path.getsize(filepath)
            frame_size = int(width * height * 1.5)  # YUV420 format
            total_frames = file_size // frame_size
            print(f"  📊 YUV file size: {file_size:,} bytes")
            print(f"  📊 Frame size: {frame_size:,} bytes")
            print(f"  📊 Total frames available: {total_frames}")
            return total_frames
        except Exception as e:
            print(f"  ❌ Error calculating frames: {e}")
            return 0
        
    def read_yuv_frames(self, filepath, width, height, max_frames=None):
        """Read YUV frames - ALL frames if max_frames is None"""
        frame_size = int(width * height * 1.5)
        y_size = width * height
        
        # If max_frames not specified, get all available frames
        if max_frames is None:
            max_frames = self.get_total_frames_in_yuv(filepath, width, height)
        
        frames = []
        with open(filepath, 'rb') as f:
            for i in tqdm(range(max_frames), desc=f"Reading ALL {max_frames} frames"):
                f.seek(i * frame_size)
                y_data = f.read(y_size)
                if len(y_data) != y_size:
                    print(f"  ⚠️ Reached end of file at frame {i}")
                    break
                frames.append(np.frombuffer(y_data, dtype=np.uint8).reshape(height, width))
        
        print(f"  ✅ Successfully read {len(frames)} frames")
        return np.array(frames)
    
    def create_hierarchical_labels_from_depth(self, cu_depth_data, frame_idx, row, col, width, height):
        """Create hierarchical labels from CU depth information"""
        # Calculate CTU position
        ctu_cols = width // 64
        ctu_rows = height // 64
        ctus_per_frame = ctu_cols * ctu_rows
        
        # Base CTU index
        ctu_idx = frame_idx * ctus_per_frame + row * ctu_cols + col
        
        # Initialize 16-element label array for this CTU
        labels = np.zeros(16, dtype=np.uint8)
        
        # Safety check for CU depth data bounds
        if ctu_idx >= len(cu_depth_data):
            return labels
            
        # 64x64 level decision (1 decision)
        depth_64 = cu_depth_data[ctu_idx] if ctu_idx < len(cu_depth_data) else 0
        labels[0] = 1 if depth_64 > 0 else 0
        
        # 32x32 level decisions (4 decisions)
        for quad in range(4):
            sub_idx = ctu_idx + quad * max(1, len(cu_depth_data) // (ctus_per_frame * 4))
            sub_idx = min(sub_idx, len(cu_depth_data) - 1)
            depth_32 = cu_depth_data[sub_idx]
            labels[1 + quad] = 1 if depth_32 > 1 else 0
        
        # 16x16 level decisions (12 decisions, positions 5-16)
        for sub in range(12):
            sub_idx = ctu_idx + sub * max(1, len(cu_depth_data) // (ctus_per_frame * 16))
            sub_idx = min(sub_idx, len(cu_depth_data) - 1)
            depth_16 = cu_depth_data[sub_idx]
            labels[4 + sub] = 1 if depth_16 > 2 else 0
            
        return labels
    
    def extract_patches_fixed(self, frames, cu_depths, width, height):
        """Extract patches with proper hierarchical labels - ALL frames"""
        ctu_cols = width // 64
        ctu_rows = height // 64
        ctus_per_frame = ctu_cols * ctu_rows
        
        all_patches = []
        all_labels = []
        
        # Process ALL available frames
        max_frames = min(len(frames), max(1, len(cu_depths) // ctus_per_frame))
        print(f"  📊 Processing {max_frames} frames ({len(frames)} YUV frames, {len(cu_depths)} CU depth values)")
        
        for frame_idx in tqdm(range(max_frames), desc="Extracting ALL patches"):
            frame = frames[frame_idx]
            for row in range(ctu_rows):
                for col in range(ctu_cols):
                    # Extract 64x64 patch
                    patch = frame[row*64:(row+1)*64, col*64:(col+1)*64]
                    
                    # Create hierarchical labels for this patch
                    hierarchical_labels = self.create_hierarchical_labels_from_depth(
                        cu_depths, frame_idx, row, col, width, height)
                    
                    all_patches.append(patch)
                    all_labels.append(hierarchical_labels)
        
        print(f"  ✅ Total patches extracted: {len(all_patches)}")
        return np.array(all_patches), np.array(all_labels)

# Initialize the FULL processor
processor = FullCPHProcessor()

In [31]:
# Cell 3: Updated Processing Function - FULL DATASET VERSION
def process_complete_dataset(dataset_type):
    """Process COMPLETE dataset - ALL available frames"""
    print(f"\n🔄 Processing COMPLETE {dataset_type} Dataset - ALL FRAMES")
    print("=" * 60)
    
    all_samples = []
    total_expected_samples = 0
    
    for resolution in processor.RESOLUTIONS:
        width, height = map(int, resolution.split('x'))
        
        # Find YUV file
        yuv_file = None
        for root, dirs, files in os.walk(processor.input_path):
            for file in files:
                if f'Intra{dataset_type}_{resolution}.yuv' in file:
                    yuv_file = os.path.join(root, file)
                    break
        
        if not yuv_file:
            print(f"❌ YUV file not found for {resolution}")
            continue
            
        print(f"\n--- {resolution} (ALL FRAMES) ---")
        
        # Calculate expected samples for this resolution
        total_frames = processor.get_total_frames_in_yuv(yuv_file, width, height)
        ctus_per_frame = (width // 64) * (height // 64)
        samples_per_qp = total_frames * ctus_per_frame
        total_expected_samples += samples_per_qp * len(processor.QP_VALUES)
        
        print(f"  📊 Expected samples for this resolution: {samples_per_qp * len(processor.QP_VALUES):,}")
        
        for qp in processor.QP_VALUES:
            # Find CU depth file
            cudepth_files = glob.glob(f'{processor.input_path}/*AI_Intra{dataset_type}_{resolution}_qp{qp}*CUDepth.dat')
            if not cudepth_files:
                print(f"❌ CU depth file not found for QP{qp}")
                continue
                
            try:
                print(f"🔄 Processing QP{qp} - ALL FRAMES...")
                
                # Read ALL frames (no limit)
                frames = processor.read_yuv_frames(yuv_file, width, height, max_frames=None)
                cu_depths = np.frombuffer(open(cudepth_files[0], 'rb').read(), dtype=np.uint8)
                
                print(f"  📊 Loaded: {len(frames)} frames, {len(cu_depths)} CU depth values")
                
                # Extract ALL patches
                patches, labels = processor.extract_patches_fixed(frames, cu_depths, width, height)
                
                print(f"  📊 Extracted: {len(patches)} patches")
                
                # Create samples with correct structure
                for patch, patch_labels in zip(patches, labels):
                    sample = np.zeros(4992, dtype=np.uint8)
                    
                    # Image data (4096 bytes)
                    sample[:4096] = patch.flatten()
                    
                    # QP value stored in padding area
                    sample[4096] = qp
                    
                    # Labels section: 832 bytes starting at position 4160
                    for qp_idx in range(52):
                        start_pos = 4160 + qp_idx * 16
                        end_pos = start_pos + 16
                        
                        if qp_idx == qp:
                            sample[start_pos:end_pos] = patch_labels
                        else:
                            sample[start_pos:end_pos] = np.zeros(16, dtype=np.uint8)
                    
                    all_samples.append(sample)
                
                print(f"  ✅ QP{qp}: {len(patches)} samples added (COMPLETE)")
                
                # Clean memory
                del frames, cu_depths, patches, labels
                gc.collect()
                
            except Exception as e:
                print(f"  ❌ Error processing QP{qp}: {e}")
                import traceback
                traceback.print_exc()
    
    if all_samples:
        all_samples = np.array(all_samples)
        np.random.shuffle(all_samples)
        
        # Save final dataset
        filename = f'AI_{dataset_type}_COMPLETE_{len(all_samples)}.dat'
        filepath = os.path.join(processor.output_path, filename)
        all_samples.tofile(filepath)
        
        size_mb = os.path.getsize(filepath) / (1024*1024)
        size_gb = size_mb / 1024
        
        print(f"\n✅ COMPLETE DATASET SAVED: {filename}")
        print(f"📊 {len(all_samples):,} samples (vs {total_expected_samples:,} expected)")
        print(f"📊 {size_mb:.1f} MB ({size_gb:.2f} GB)")
        print(f"🔍 Sample size: {all_samples.shape[1]} bytes (expected: 4992)")
        
        # Verify sample structure
        sample = all_samples[0]
        print(f"🔍 Sample verification:")
        print(f"   Image data range: {sample[:4096].min()}-{sample[:4096].max()}")
        print(f"   QP value: {sample[4096]}")
        print(f"   Labels range: {sample[4160:].min()}-{sample[4160:].max()}")
        
        return filepath
    
    return None


In [32]:
# Cell 4: Process COMPLETE datasets
print("🚀 PROCESSING COMPLETE DATASETS - ALL FRAMES")
print("⚠️ WARNING: This will take much longer and generate much larger files!")
print("=" * 60)

# Process each dataset completely
datasets_to_process = ['Test', 'Valid']  # Start with Test first to check

for dataset_type in datasets_to_process:
    print(f"\n🎯 Processing COMPLETE {dataset_type} Dataset")
    result_file = process_complete_dataset(dataset_type)
    
    if result_file:
        print(f"✅ COMPLETE {dataset_type} dataset saved: {os.path.basename(result_file)}")
    else:
        print(f"❌ COMPLETE {dataset_type} dataset failed")
    
    print(f"🔋 Memory cleanup...")
    gc.collect()

🚀 PROCESSING COMPLETE DATASETS - ALL FRAMES
⚠️ WARNING: This will take much longer and generate much larger files!

🎯 Processing COMPLETE Test Dataset

🔄 Processing COMPLETE Test Dataset - ALL FRAMES

--- 768x512 (ALL FRAMES) ---
  📊 YUV file size: 29,491,200 bytes
  📊 Frame size: 589,824 bytes
  📊 Total frames available: 50
  📊 Expected samples for this resolution: 19,200
🔄 Processing QP22 - ALL FRAMES...
  📊 YUV file size: 29,491,200 bytes
  📊 Frame size: 589,824 bytes
  📊 Total frames available: 50


Reading ALL 50 frames: 100%|██████████| 50/50 [00:00<00:00, 4847.45it/s]


  ✅ Successfully read 50 frames
  📊 Loaded: 50 frames, 76800 CU depth values
  📊 Processing 50 frames (50 YUV frames, 76800 CU depth values)


Extracting ALL patches: 100%|██████████| 50/50 [00:00<00:00, 300.12it/s]

  ✅ Total patches extracted: 4800


...

Reading ALL 25 frames: 100%|██████████| 25/25 [00:00<00:00, 271.03it/s]

  ✅ Successfully read 25 frames


  📊 Loaded: 25 frames, 1570800 CU depth values
  📊 Processing 25 frames (25 YUV frames, 1570800 CU depth values)


Extracting ALL patches: 100%|██████████| 25/25 [00:03<00:00,  7.31it/s]


  ✅ Total patches extracted: 98175
  📊 Extracted: 98175 patches
  ✅ QP37: 98175 samples added (COMPLETE)

✅ COMPLETE DATASET SAVED: AI_Valid_COMPLETE_575700.dat
📊 575,700 samples (vs 575,700 expected)
📊 2740.8 MB (2.68 GB)
🔍 Sample size: 4992 bytes (expected: 4992)
🔍 Sample verification:
   Image data range: 119-215
   QP value: 27
   Labels range: 0-1
✅ COMPLETE Valid dataset saved: AI_Valid_COMPLETE_575700.dat
🔋 Memory cleanup...


In [ ]:
# Cell 4: Process COMPLETE datasets
print("🚀 PROCESSING COMPLETE DATASETS - ALL FRAMES")
print("⚠️ WARNING: This will take much longer and generate much larger files!")
print("=" * 60)

# Process each dataset completely
datasets_to_process = ['Train']  # Start with Test first to check

for dataset_type in datasets_to_process:
    print(f"\n🎯 Processing COMPLETE {dataset_type} Dataset")
    result_file = process_complete_dataset(dataset_type)
    
    if result_file:
        print(f"✅ COMPLETE {dataset_type} dataset saved: {os.path.basename(result_file)}")
    else:
        print(f"❌ COMPLETE {dataset_type} dataset failed")
    
    print(f"🔋 Memory cleanup...")
    gc.collect()

In [ ]:
# Cell 7: Verification and Results
print("\n📊 Final Results (FIXED VERSION):")
print("=" * 60)

total_samples = 0
files = ['Test', 'Valid', 'Train']

for dataset in files:
    for file in os.listdir('/kaggle/working'):
        if f'AI_{dataset}_' in file and file.endswith('.dat'):
            filepath = f'/kaggle/working/{file}'
            size_mb = os.path.getsize(filepath) / (1024*1024)
            num_samples = os.path.getsize(filepath) // 4992  # Note: 4992 bytes per sample now
            
            print(f"{dataset:>5}: {file}")
            print(f"        {size_mb:>7.1f} MB | {num_samples:>8,} samples | 4992 bytes/sample")
            total_samples += num_samples
            break

print("=" * 60)
print(f"TOTAL: {total_samples:,} samples")
print(f"✅ All samples now have correct 4992-byte structure")
print(f"✅ Hierarchical labels properly implemented")
print(f"💾 Ready for download from Output tab!")

# Verify a sample from each dataset
print(f"\n🔍 Sample Verification:")
for dataset in files:
    for file in os.listdir('/kaggle/working'):
        if f'AI_{dataset}_' in file and file.endswith('.dat'):
            filepath = f'/kaggle/working/{file}'
            
            # Read first sample
            with open(filepath, 'rb') as f:
                sample = np.frombuffer(f.read(4992), dtype=np.uint8)
                
            print(f"{dataset}: Image[{sample[:4096].min()}-{sample[:4096].max()}], "
                  f"QP={sample[4096]}, Labels[{sample[4160:].min()}-{sample[4160:].max()}]")
            break


In [26]:
# import os
# import shutil

# working_dir = '/kaggle/working'

# # List all files and directories in /kaggle/working
# for filename in os.listdir(working_dir):
#     file_path = os.path.join(working_dir, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.remove(file_path)  # Remove file or link
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)  # Remove directory and all its contents
#         print(f'Deleted: {file_path}')
#     except Exception as e:
#         print(f'Failed to delete {file_path}. Reason: {e}')
